In [62]:
pip install langdetect


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import xml.etree.ElementTree as ET
import os
import re
from langdetect import detect
#from langdetect.lang_detect_exception import LangDetectException
import json

In [3]:
# Définition des dossiers
data_folder = "data"
folders = ["FR", "IT", "NL", "UK", "GE"]
input_subfolder = "corrigées"
output_subfolder = "corrigées_xml"

### Création de fichiers `.xml` structurés pour les transcriptions manuelles (avec les énoncés des apprenants et ses versions correctes en polonais)

In [4]:
def clean_text(text):
    """Fonction pour supprimer les caractères '#' (qui marquent des longues pauses)"""
    if text:
        return text.replace("#", "")
    return text

In [5]:
#LANGUES_A_EXCLURE = {"fr", "it", "nl", "de", "en"}

#mots_detectés_global = {lang: set() for lang in LANGUES_A_EXCLURE}

#def filter_text(text):
#    """
#    Supprime les mots détectés comme appartenant aux langues suivantes :
#    français, italien, anglais, allemand et néerlandais.
#    Ajoute les mots détectés à un dictionnaire global par langue.
#    """
#    mots = re.findall(r"\w+", text)
#    mots_gardés = []

#    for mot in mots:
#        try:
#            langue = detect(mot.lower())
#        except LangDetectException:
#            langue = "unk"
#        if langue in LANGUES_A_EXCLURE:
#            mots_detectés_global[langue].add(mot)
#        else:
#            mots_gardés.append(mot)

#    return " ".join(mots_gardés)

In [6]:
def process_eaf_files(data_folder, folders, input_subfolder="corrigées", output_subfolder="corrigées_xml"):
    """
    Cette fonction parcourt les fichiers .eaf dans les dossiers spécifiés,
    extrait les annotations des apprenants (*STU) et les annotations avec la version correcte en polonais (%pol),
    puis génère un fichier XML structuré pour chaque fichier .eaf.
    
    Args:
        data_folder (str): Chemin du dossier principal contenant les sous-dossiers.
        folders (list): Liste des sous-dossiers correspondant aux langues (FR, IT, NL, UK, GE).
        input_subfolder (str): Nom du sous-dossier contenant les fichiers .eaf.
        output_subfolder (str): Nom du sous-dossier où stocker les fichiers XML générés.
    """
    for folder in folders:
        input_path = os.path.join(data_folder, folder, input_subfolder)
        output_path = os.path.join(data_folder, folder, output_subfolder)
        os.makedirs(output_path, exist_ok=True)

        for filename in os.listdir(input_path):
            if filename.endswith(".eaf"):
                input_file = os.path.join(input_path, filename)
                output_file = os.path.join(output_path, filename.replace(".eaf", ".xml"))

                # Charger le fichier EAF
                tree = ET.parse(input_file)
                root = tree.getroot()

                time_slots = {}
                annotations_stu = []
                annotations_pol = []
                student_id = None

                # Extraire les time slots
                for time_slot in root.find("TIME_ORDER").findall("TIME_SLOT"):
                    time_slots[time_slot.attrib["TIME_SLOT_ID"]] = int(time_slot.attrib["TIME_VALUE"])

                # Extraire les annotations
                for tier in root.findall("TIER"):
                    tier_id = tier.attrib["TIER_ID"]
                    for annotation in tier.findall("ANNOTATION"):
                        alignable_annotation = annotation.find("ALIGNABLE_ANNOTATION")
                        start = time_slots[alignable_annotation.attrib["TIME_SLOT_REF1"]]
                        end = time_slots[alignable_annotation.attrib["TIME_SLOT_REF2"]]
                        text = alignable_annotation.find("ANNOTATION_VALUE").text
                        text = clean_text(text)  # Supprimer les '#'
                        if tier_id == "*STU":
                            if student_id is None:
                                student_id = text  # Premier segment *STU = numéro d'apprenant
                            else:
                                annotations_stu.append((start, end, text))
                        elif tier_id == "%pol":
                            annotations_pol.append((start, end, text))

                # Trier par ordre chronologique
                annotations_stu.sort()
                annotations_pol.sort()

                # Créer un nouvel arbre XML
                new_root = ET.Element("annotations")

                if student_id:
                    student_elem = ET.SubElement(new_root, "apprenant_id")
                    student_elem.text = student_id

                    country_elem = ET.SubElement(new_root, "pays")
                    country_elem.text = folder

                for i, (stu, pol) in enumerate(zip(annotations_stu, annotations_pol), start=1):
                    pair = ET.SubElement(new_root, "énoncé", id=str(i))
                    stu_elem = ET.SubElement(pair, "STU")
                    stu_elem.text = stu[2]
                    pol_elem = ET.SubElement(pair, "POL")
                    pol_elem.text = pol[2]

                # Sauvegarder le fichier
                new_tree = ET.ElementTree(new_root)
                new_tree.write(output_file, encoding="UTF-8", xml_declaration=True)

process_eaf_files(data_folder, folders)

In [7]:
#mots_detectés_global = {lang: sorted(list(mots)) for lang, mots in mots_detectés_global.items() if mots}
#print(mots_detectés_global)

### Nettoyage des transcriptions automatiques et l'intégration à l'ensemble des données

In [8]:
def clean_text(text):
    """
    Fonction pour supprimer la ponctuation (sauf les tirets), convertir toutes les lettres en minuscules, 
    supprimer les retours à la ligne et enlever les espaces en début/fin.
    """
    # Remplacer les retours à la ligne par un espace
    text = text.replace("\n", " ")

    # Supprimer la ponctuation sauf les tirets et convertir toutes les lettres en minuscules
    cleaned_text = ''.join([char.lower() if char.isalnum() or char.isspace() or char == '-' else '' for char in text])

    # Supprimer les espaces en début et fin
    cleaned_text = cleaned_text.strip()

    return cleaned_text

In [9]:
def process_automatiques_folder(data_folder, folders, input_subfolder="automatiques"):
    """Traite les transcriptions automatiques et les nettoie."""
    cleaned_data = []

    for folder in folders:
        input_path = os.path.join(data_folder, folder, input_subfolder)

        if not os.path.exists(input_path):
            print(f"Le dossier {input_path} n'existe pas.")
            continue

        for filename in os.listdir(input_path):
            if filename.endswith(".txt"):
                input_file = os.path.join(input_path, filename)

                with open(input_file, 'r', encoding='utf-8') as file:
                    content = file.read()

                try:
                    langue_detectee = detect(content)
                except Exception:
                    langue_detectee = "unknown"

                cleaned_content = clean_text(content)
                file_number = filename.replace(".txt", "")

                if langue_detectee == 'pl':
                    cleaned_data.append({
                        'dossier': folder,
                        'fichier': file_number,
                        'transcription automatique': cleaned_content
                    })
                else:
                    cleaned_data.append({
                        'dossier': folder,
                        'fichier': file_number,
                        'transcription_automatique_non_pl': cleaned_content
                    })

    return cleaned_data

In [10]:
def clean_double_spaces(text):
    """Fonction pour supprimer les espaces doubles."""
    return ' '.join(text.split())

In [11]:
def enrich_cleaned_data(cleaned_data, data_folder, folders, xml_subfolder="corrigées_xml"):
    enriched_data = []

    for folder in folders:
        xml_path = os.path.join(data_folder, folder, xml_subfolder)
        xml_data = {}

        for xml_file in os.listdir(xml_path):
            if xml_file.endswith(".xml"):
                file_number = xml_file.replace(".xml", "")
                file_path = os.path.join(xml_path, xml_file)

                tree = ET.parse(file_path)
                root = tree.getroot()

                apprenant_id = root.find("apprenant_id").text if root.find("apprenant_id") is not None else ""
                pays = root.find("pays").text if root.find("pays") is not None else ""

                segment_dict = {}
                for i, enonce in enumerate(root.findall("énoncé"), start=1):
                    stu_text = enonce.find("STU").text or ""
                    pol_text = enonce.find("POL").text or ""

                    stu_text = clean_double_spaces(stu_text)
                    pol_text = clean_double_spaces(pol_text)

                    segment_dict[f"transcription manuelle ({i})"] = stu_text
                    segment_dict[f"version correcte en polonais ({i})"] = pol_text

                xml_data[file_number] = {
                    "apprenant_id": apprenant_id,
                    "pays": pays,
                    **segment_dict
                }

        for entry in cleaned_data:
            file_number = entry["fichier"]

            if file_number in xml_data:
                base_info = {
                    "numéro d'apprenant": xml_data[file_number]["apprenant_id"],
                    "pays": xml_data[file_number]["pays"]
                }

                # Ajouter la bonne transcription
                if "transcription automatique" in entry:
                    base_info["transcription automatique"] = entry["transcription automatique"]
                elif "transcription_automatique_non_pl" in entry:
                    base_info["transcription_automatique_non_pl"] = entry["transcription_automatique_non_pl"]

                segments = {k: v for k, v in xml_data[file_number].items() if k not in ["apprenant_id", "pays"]}
                enriched_entry = {**base_info, **segments}
                enriched_data.append(enriched_entry)

    return enriched_data

# Appels des fonctions principales
cleaned_data = process_automatiques_folder(data_folder, folders)
enriched_data = enrich_cleaned_data(cleaned_data, data_folder, folders)

In [12]:
# Sauvegarde du résultat final
output_json_path = os.path.join(data_folder, "../transcriptions_ordonnées.json")

with open(output_json_path, mode="w", encoding="utf-8") as jsonfile:
    json.dump(enriched_data, jsonfile, ensure_ascii=False, indent=4)


In [13]:
# Affichage des exemples dont la transcription automatique n'est pas en polonais
non_pl_examples = [
    (entry["numéro d'apprenant"], entry["pays"])
    for entry in enriched_data
    if "transcription_automatique_non_pl" in entry
]

# Affichage des résultats
print("Transcriptions automatiques NON polonaises")
for apprenant_id, pays in non_pl_examples:
    print(f"Apprenant : {apprenant_id}, Pays : {pays}")

print(f"\nNombre total d'exemples non polonais : {len(non_pl_examples)}")


Transcriptions automatiques NON polonaises
Apprenant : 1114, Pays : FR
Apprenant : 1105, Pays : FR
Apprenant : 5114, Pays : IT
Apprenant : 5105, Pays : IT
Apprenant : 5120, Pays : IT
Apprenant : 5109, Pays : IT
Apprenant : 3103, Pays : UK
Apprenant : 3118, Pays : UK
Apprenant : 3101, Pays : UK
Apprenant : 3104, Pays : UK
Apprenant : 3105, Pays : UK
Apprenant : 3106, Pays : UK
Apprenant : 4101, Pays : GE
Apprenant : 4118, Pays : GE
Apprenant : 4108, Pays : GE

Nombre total d'exemples non polonais : 15
